In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from spp.convex_sets import Singleton, Polyhedron
from spp.convex_functions import SquaredTwoNorm
from spp.graph import GraphOfConvexSets
from spp.shortest_path import ShortestPathProblem

In [ ]:
scale = 4

v1 = np.array([[1, 1], [2, 1], [2, 2], [1, 2]]) - np.array([0, .01])
v2 = np.array([[1, -1], [2, -1], [2, -2], [1, -2]])
v3 = np.array([[3, 1], [4, 1], [4, 2], [3, 2]]) - np.array([0, .01])
v4 = np.array([[3, -1], [4, -1], [4, -2], [3, -2]])

# convex sets
sets = (
    Singleton((0, 0)),
    Singleton((5, 0)),
    Polyhedron.from_vertices(v1),
    Polyhedron.from_vertices(v2),
    Polyhedron.from_vertices(v3),
    Polyhedron.from_vertices(v4),
)

# label for the vertices
vertices = ['s', 't', '1', '2', '3', '4']

# add convex sets to the graph
G = GraphOfConvexSets()
G.add_sets(sets, vertices)
G.set_source('s')
G.set_target('t')

# edges
H = np.hstack((np.eye(2), -np.eye(2)))
l = SquaredTwoNorm(H)
edges = {
    's': ('1', '2'),
    '1': ('3',),
    '2': ('1', '4'),
    '3': ('4', 't'),
    '4': ('t',),
}
for u, vs in edges.items():
    for v in vs:
        G.add_edge(u, v, l)

# scale sets
G_scaled = deepcopy(G)
G_scaled.scale(.5 * scale)
        
# solve spp
spp = ShortestPathProblem(G_scaled, relaxation=0)
sol = spp.solve()
        
# draw convex sets and edges
plt.figure(figsize=(3,3))
plt.axis('off')
plt.xlim([-.2, 5.2])
plt.ylim([-2.6, 2.6])

G_scaled.draw_sets()
G_scaled.draw_edges()
G_scaled.draw_path(sol.primal.phi, sol.primal.x, color='r')
# plt.savefig(f'reduction_{scale}.pdf', bbox_inches='tight')